In [1]:
using MPSKitModels
using TensorKit
using TensorOperations
using LinearAlgebra 
include("Potts-Operators & Hamiltonian.jl")
using SymGroupRepresentations
# import Pkg
# Pkg.activate(".")
# Pkg.add(path ="https://github.com/Yue-Zhengyuan/SymGroupRepresentations.jl")

In [2]:
L = 6
Q = 5
J=1
h=1
lambda=0.079 + 0.060im
_,_,W = weyl_heisenberg_matrices(Q)
P   = TensorMap(W,ℂ^Q←ℂ^Q)
lat = FiniteChain(L)
dat0 = reshape((P*sum((-h * potts_spin_shift(; q = Q,k=j)') for j in 1:1:Q-1)*P').data, (Q,Q))
dat1 = reshape(((P ⊗ P)*sum((-J * potts_phase(; q=Q,k=j)') for j in 1:1:Q-1)*(P' ⊗ P')).data, (Q,Q,Q,Q))
dat2 = reshape(((P ⊗ P) * sum(sum(potts_phase_shift_combined(;q=Q,k=l,p=j)' for l in 1:1:Q-1) for j in 1:1:Q-1) *(P'⊗P')).data, (Q,Q,Q,Q))
println("done")

done


In [3]:
Vp = Vect[S5Irrep](S5Irrep([4,1])=>1, S5Irrep([5])=>1)
# Vp = Vect[S5Irrep](ob=>120)
H0 = @mpoham (sum(TensorMap(dat0,Vp←Vp){i} for i in vertices(lat)[1:(end)])) ### Potts
H1 = @mpoham (sum(TensorMap(dat1, Vp⊗Vp←Vp⊗Vp){i,i+1}  for i in vertices(lat)[1:(end-1)]) + TensorMap(dat1, Vp⊗Vp←Vp⊗Vp){vertices(lat)[end],vertices(lat)[1]}) ##¨Potts with BC
H2 =  @mpoham lambda'* sum(TensorMap(dat2,Vp⊗Vp←Vp⊗Vp){i,i+1} for i in vertices(lat)[1:(end - 1)]) + lambda' * TensorMap(dat2, Vp⊗Vp←Vp⊗Vp){vertices(lat)[end],vertices(lat)[1]} ###Extra term
ham = H0+H1+H2

D = 50
println(Vp)
ψ = FiniteMPS(L,Vp,Vect[S5Irrep](S5Irrep([4,1])=>D, S5Irrep([5])=>D))
t = time()
(ψ, envir , delta)   = find_groundstate(ψ, ham, DMRG(maxiter = 500,tol=1e-6, eigalg =MPSKit.Defaults.alg_eigsolve(; ishermitian=false)))
println(time()-t)
println(expectation_value(ψ,ham))

ArgumentError: ArgumentError: Data has non-zero elements at incompatible positions

In [4]:
Vp = Vect[ZNIrrep{Q}](sector=>1 for sector in 0:Q-1)
# Vp = Vect[S5Irrep](ob=>120)
D = 50
H0 = @mpoham (sum(TensorMap(dat0,Vp←Vp){i} for i in vertices(lat)[1:(end)])) ### Potts
H1 = @mpoham (sum(TensorMap(dat1, Vp⊗Vp←Vp⊗Vp){i,i+1}  for i in vertices(lat)[1:(end-1)]) + TensorMap(dat1, Vp⊗Vp←Vp⊗Vp){vertices(lat)[end],vertices(lat)[1]}) ##¨Potts with BC
H2 =  @mpoham lambda'* sum(TensorMap(dat2,Vp⊗Vp←Vp⊗Vp){i,i+1} for i in vertices(lat)[1:(end - 1)]) + lambda' * TensorMap(dat2, Vp⊗Vp←Vp⊗Vp){vertices(lat)[end],vertices(lat)[1]} ###Extra term
ham = H0+H1+H2
J=1
h=1
lambda=0.079 + 0.060im
H0 = @mpoham (sum(TensorMap(dat0,Vp←Vp){i} for i in vertices(lat)[1:(end)])) ### Potts
ψ = FiniteMPS(L,Vp,Vect[ZNIrrep{Q}](0=>D,1=>D,2=>D,3=>D,4=>D))
t = time()
(ψ, envir , delta)   = find_groundstate(ψ, ham, DMRG(maxiter = 500,tol=1e-6, eigalg =MPSKit.Defaults.alg_eigsolve(; ishermitian=false)))
println(time()-t)
println(expectation_value(ψ,ham))

┌ Info: DMRG init:	obj = -2.062478166199e+01 +2.051838955783e-01im	err = 2.1794e-01
└ @ MPSKit C:\Users\vicva\.julia\packages\MPSKit\EfZBD\src\algorithms\groundstate\dmrg.jl:35
┌ Info: DMRG   1:	obj = -2.554173513766e+01 -1.717174893798e+00im	err = 2.6157152466e-06	time = 7.78 sec
└ @ MPSKit C:\Users\vicva\.julia\packages\MPSKit\EfZBD\src\algorithms\groundstate\dmrg.jl:57
┌ Info: DMRG conv 2:	obj = -2.554173349375e+01 -1.717175090202e+00im	err = 5.1511802498e-11	time = 8.76 sec
└ @ MPSKit C:\Users\vicva\.julia\packages\MPSKit\EfZBD\src\algorithms\groundstate\dmrg.jl:51


96.21500015258789
-25.54173349375127 - 1.7171750902022371im
